In [1]:
import numpy as np
import xarray as xr
import time 
import os
from netCDF4 import Dataset
from pyproj import CRS
import rasterio
import rioxarray
from rasterio.enums import Resampling
from sklearn.preprocessing import StandardScaler

In [2]:
filename_biostates_2019 = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/BioStates_2019.nc'
biostates_2019 = xr.open_dataset(filename_biostates_2019)

In [3]:
filename_biostates_2022 = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/BioStates_2022.nc'
biostates_2022 = xr.open_dataset(filename_biostates_2022)

In [19]:
print(list(biostates_2019.variables))

['time', 'grid_mapping', 'LayerDepths', 'xc', 'yc', 'zc', 'depth', 'DXxDYy', 'nitrate', 'silicate', 'ammonium', 'diatoms', 'flagellates', 'ciliates', 'HNANO', 'bacteria', 'calanus_finmarchicus', 'calanus_glacialis', 'detritus_slow', 'detritus_fast', 'DOC', 'cDOM', 'silicate_detritus', 'sediment_Si', 'sediment_N']


In [ ]:
biostates = xr.concat([biostates_2019, biostates_2022], dim='time')

In [ ]:
print(biostates['LayerDepths'])